In [10]:
# Ignore errors resulting from matplotlib's fig.show(). This causes
# a warning when run from a notebook, but is still required if the
# scripts are run from the CL
import warnings
warnings.simplefilter("ignore", UserWarning)

#### Comparison Scripts

The `compare_sf` script can read structure factors from both Euphonic `StructureFactor` objects in a `.json` file, and Ab2tds structure factors written to a `alongthelineF.dat` file, and calculates the mean, min and max absolute/relative errors. It also applies a scaling factor, and sums the structure factors over degenerate modes. See the script for details on how it does this.

The `compare_sqw` script can read S(Q,w) from both Euphonic `Spectrum2D` objects in a `.json` file, and OClimax S(Q,w) written to a `.csv` file. It also calculates the mean, min and max absolute/relative errors, after applying a scaling factor. See the script for details.

These scripts can be run for all the cuts for different materials to give an idea of how well they compare to experimental data.

### Structure Factor Comparison

#### Comparing with eigenvalues/vectors read from .phonon

Lets start with comparing the Ab2tds structure factors with Euphonic's structure factors that have been calculated with frequencies/eigenvectors read from a `.phonon` file:

In [2]:
from compare_sf import main as compare_sf_main
from compare_sqw import main as compare_sqw_main

quartz_cuts = ['2ph_m4_0_qe', '30L_qe']
for cut in quartz_cuts:
    compare_sf_main(['--sf1', '../../quartz/' + cut + '/euphonic/sf_phonons_100K.json',
                     '--sf2', '../../quartz/' + cut + '/ab2tds/alongthelineF_100K.dat'])

lzo_cuts = ['kagome_qe', 'hh2_qe']
for cut in lzo_cuts:
    compare_sf_main(['--sf1', '../../lzo/' + cut + '/euphonic/sf_phonons_300K.json',
                     '--sf2', '../../lzo/' + cut + '/ab2tds/alongthelineF_300K.dat'])



Results for ../../quartz/2ph_m4_0_qe/euphonic/sf_phonons_100K.json ../../quartz/2ph_m4_0_qe/ab2tds/alongthelineF_100K.dat
Absolute Error - mean: 4.6726310138278893e-08 max: 8.771275944425769e-05 min: 1.5503279671338899e-18
Relative Error - mean: 0.0024341073499717556 max: 0.03879472322547094 min: 5.177800648450457e-08

Results for ../../quartz/30L_qe/euphonic/sf_phonons_100K.json ../../quartz/30L_qe/ab2tds/alongthelineF_100K.dat
Absolute Error - mean: 3.2506643807483944e-10 max: 4.575939232826648e-07 min: 1.2786292156330635e-17
Relative Error - mean: 0.0011989250980394372 max: 0.02014565391144346 min: 4.2267131782193544e-07

Results for ../../lzo/kagome_300K_qe/euphonic/sf_phonons_300K.json ../../lzo/kagome_300K_qe/ab2tds/alongthelineF_300K.dat
Absolute Error - mean: 2.6061351104422008e-06 max: 0.0020175869410402614 min: 3.8914142380054763e-16
Relative Error - mean: 0.016305304544738157 max: 0.9587154539402496 min: 2.389378872732815e-05

Results for ../../lzo/hh2_qe/euphonic/sf_phonon

The errors for the Quartz cuts are very small - showing mean relative errors of 0.2% and 0.1%, with maximum errors of only 4% and 2%, which is satisfactory.

However, the errors for LZO seem much higher - one mean error is 1.6% and maximum errors are 95% and 25%. Lets investigate the `kagome_300K_qe` cut further, first by running with both Euphonic and Ab2tds at a lower temperature - 100K instead of 300K:

In [12]:
cut = 'kagome_qe'
compare_sf_main(['--sf1', '../../lzo/' + cut + '/euphonic/sf_phonons_100K.json',
                 '--sf2', '../../lzo/' + cut + '/ab2tds/alongthelineF_100K.dat'])


Results for ../../lzo/kagome_qe/euphonic/sf_phonons_100K.json ../../lzo/kagome_qe/ab2tds/alongthelineF_100K.dat
Absolute Error - mean: 4.2808383788196183e-07 max: 0.00032312899714653787 min: 5.826038737109541e-17
Relative Error - mean: 0.005977585584588909 max: 1.3488109129264791 min: 3.7099340476906613e-07


This reduces the mean error to 0.5% - so the temperature does have an effect (although the maximum relative error is actually higher!). The difference then is likely to be down to the Debye-Waller factor (Ab2tds doesn't calculate the Bose population factor). When calculating the Debye-Waller factor Ab2tds does some symmetry operations to unfold the input grid q-points to the whole BZ - this could be a source of discrepancy.

But the overall relative errors might be giving a pessimistic picture of the comparison. Looking at the 300K `kagome_qe` data again, we can figure out which q-point is causing the largest relative error and take a closer look.

In [15]:
import numpy as np
from compare_sf import get_sf, get_degenerate_modes, calc_sf_sum
from util import get_scaling

eusf, eufr, euq = get_sf('../../lzo/kagome_qe/euphonic/sf_phonons_300K.json')
absf, abfr, abq = get_sf('../../lzo/kagome_qe/ab2tds/alongthelineF_300K.dat')

# We need to sum over degenerate phonon modes and scale the results to get a good comparison
dg_modes = get_degenerate_modes(eufr)
eusf_sum = calc_sf_sum(dg_modes, eusf)
absf_sum = calc_sf_sum(dg_modes, absf)
scale = get_scaling(eusf_sum, absf_sum)